# **Benchmark Dataset Loader**

Carga y gestión de datasets de benchmarking para coloración de grafos:
- Ejemplos simples (sintéticos)
- SNAP datasets (redes reales)
- DIMACS benchmarks (instancias clásicas)

In [1]:
%run librerias.ipynb

C:\Users\ismae\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cpu
Librerías cargadas exitosamente


## **1. GENERADORES DE GRAFOS SIMPLES**

In [ ]:
def generar_grafo_simple(tipo, n=50, **kwargs):
    """
    Genera grafos sintéticos para pruebas básicas.
    
    Tipos disponibles:
    - 'random': Erdős-Rényi G(n,p)
    - 'complete': Grafo completo K_n
    - 'cycle': Ciclo C_n
    - 'bipartite': Grafo bipartito completo
    - 'tree': Árbol aleatorio
    - 'wheel': Grafo rueda
    - 'petersen': Grafo de Petersen
    """
    if tipo == 'random':
        p = kwargs.get('p', 0.1)
        G = nx.erdos_renyi_graph(n, p)
        nombre = f"Random_n{n}_p{p}"
    elif tipo == 'complete':
        G = nx.complete_graph(n)
        nombre = f"Complete_K{n}"
    elif tipo == 'cycle':
        G = nx.cycle_graph(n)
        nombre = f"Cycle_C{n}"
    elif tipo == 'bipartite':
        m = kwargs.get('m', n//2)
        G = nx.complete_bipartite_graph(n, m)
        nombre = f"Bipartite_K{n}_{m}"
    elif tipo == 'tree':
        G = nx.random_labeled_tree(n)
        nombre = f"Tree_n{n}"
    elif tipo == 'wheel':
        G = nx.wheel_graph(n)
        nombre = f"Wheel_W{n}"
    elif tipo == 'petersen':
        G = nx.petersen_graph()
        nombre = "Petersen"
    else:
        raise ValueError(f"Tipo de grafo desconocido: {tipo}")
    
    return G, nombre

## **2. PARSER FORMATO DIMACS**

In [3]:
def cargar_dimacs(filepath):
    """
    Carga un grafo en formato DIMACS.
    
    Formato:
    c comentarios
    p edge <num_vertices> <num_aristas>
    e <u> <v>
    """
    G = nx.Graph()
    
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith('c'):
                    continue
                
                parts = line.split()
                
                if parts[0] == 'p':
                    num_vertices = int(parts[2])
                    G.add_nodes_from(range(1, num_vertices + 1))
                
                elif parts[0] == 'e':
                    u, v = int(parts[1]), int(parts[2])
                    G.add_edge(u, v)
        
        mapping = {node: i for i, node in enumerate(sorted(G.nodes()))}
        G = nx.relabel_nodes(G, mapping)
        
        return G
    
    except FileNotFoundError:
        print(f"Archivo no encontrado: {filepath}")
        return None

## **3. PARSER FORMATO SNAP**

In [4]:
def cargar_snap(filepath, directed=False):
    """
    Carga un grafo en formato SNAP (edge list).
    
    Formato:
    # comentarios
    <u> <v>
    """
    if directed:
        G = nx.DiGraph()
    else:
        G = nx.Graph()
    
    try:
        with open(filepath, 'r') as f:
            for line in f:
                line = line.strip()
                if not line or line.startswith('#'):
                    continue
                
                parts = line.split()
                if len(parts) >= 2:
                    u, v = int(parts[0]), int(parts[1])
                    if u != v:
                        G.add_edge(u, v)
        
        if directed:
            G = G.to_undirected()
        
        mapping = {node: i for i, node in enumerate(sorted(G.nodes()))}
        G = nx.relabel_nodes(G, mapping)
        
        return G
    
    except FileNotFoundError:
        print(f"Archivo no encontrado: {filepath}")
        return None

## **4. GENERADOR DE BENCHMARKS SINTÉTICOS COMPLEJOS**

In [5]:
def generar_benchmark_complejo(tipo, n=100, **kwargs):
    """
    Genera grafos sintéticos más complejos para evaluación.
    
    Tipos:
    - 'scale_free': Red libre de escala (Barabási-Albert)
    - 'small_world': Mundo pequeño (Watts-Strogatz)
    - 'random_regular': Grafo k-regular aleatorio
    - 'geometric': Grafo geométrico aleatorio
    - 'powerlaw_cluster': Clustering con ley de potencias
    """
    if tipo == 'scale_free':
        m = kwargs.get('m', 3)
        G = nx.barabasi_albert_graph(n, m)
        nombre = f"ScaleFree_n{n}_m{m}"
    
    elif tipo == 'small_world':
        k = kwargs.get('k', 4)
        p = kwargs.get('p', 0.3)
        G = nx.watts_strogatz_graph(n, k, p)
        nombre = f"SmallWorld_n{n}_k{k}_p{p}"
    
    elif tipo == 'random_regular':
        d = kwargs.get('d', 3)
        G = nx.random_regular_graph(d, n)
        nombre = f"RandomRegular_n{n}_d{d}"
    
    elif tipo == 'geometric':
        radius = kwargs.get('radius', 0.2)
        G = nx.random_geometric_graph(n, radius)
        nombre = f"Geometric_n{n}_r{radius}"
    
    elif tipo == 'powerlaw_cluster':
        m = kwargs.get('m', 3)
        p = kwargs.get('p', 0.3)
        G = nx.powerlaw_cluster_graph(n, m, p)
        nombre = f"PowerlawCluster_n{n}_m{m}_p{p}"
    
    else:
        raise ValueError(f"Tipo de benchmark desconocido: {tipo}")
    
    return G, nombre

## **5. SUITE DE BENCHMARKS PREDEFINIDA**

In [6]:
def cargar_suite_benchmarks(nivel='simple'):
    """
    Carga una suite de benchmarks predefinida.
    
    Niveles:
    - 'simple': Grafos pequeños y básicos
    - 'medio': Grafos sintéticos complejos medianos
    - 'complejo': Grafos grandes y desafiantes
    - 'completo': Todos los niveles
    """
    benchmarks = []
    
    if nivel in ['simple', 'completo']:
        benchmarks.extend([
            generar_grafo_simple('petersen'),
            generar_grafo_simple('cycle', n=20),
            generar_grafo_simple('wheel', n=15),
            generar_grafo_simple('complete', n=10),
            generar_grafo_simple('bipartite', n=10, m=10),
            generar_grafo_simple('tree', n=30),
            generar_grafo_simple('random', n=30, p=0.15),
        ])
    
    if nivel in ['medio', 'completo']:
        benchmarks.extend([
            generar_benchmark_complejo('scale_free', n=100, m=3),
            generar_benchmark_complejo('small_world', n=100, k=6, p=0.3),
            generar_benchmark_complejo('random_regular', n=100, d=5),
            generar_benchmark_complejo('geometric', n=100, radius=0.15),
            generar_benchmark_complejo('powerlaw_cluster', n=100, m=4, p=0.3),
            generar_grafo_simple('random', n=100, p=0.1),
        ])
    
    if nivel in ['complejo', 'completo']:
        benchmarks.extend([
            generar_benchmark_complejo('scale_free', n=500, m=5),
            generar_benchmark_complejo('small_world', n=500, k=8, p=0.2),
            generar_benchmark_complejo('random_regular', n=500, d=10),
            generar_grafo_simple('random', n=500, p=0.05),
            generar_benchmark_complejo('powerlaw_cluster', n=500, m=5, p=0.2),
        ])
    
    return benchmarks

## **6. ESTADÍSTICAS DE BENCHMARK**

In [7]:
def estadisticas_grafo(G, nombre):
    """
    Calcula estadísticas detalladas de un grafo.
    """
    n = G.number_of_nodes()
    m = G.number_of_edges()
    
    if n == 0:
        return None
    
    densidad = nx.density(G)
    grados = dict(G.degree())
    grado_promedio = np.mean(list(grados.values()))
    grado_max = max(grados.values())
    grado_min = min(grados.values())
    
    num_componentes = nx.number_connected_components(G)
    
    if m > 0:
        clustering_promedio = nx.average_clustering(G)
    else:
        clustering_promedio = 0.0
    
    cota_superior_cromatica = grado_max + 1
    
    if nx.is_bipartite(G):
        numero_cromatico_teorico = 2
    elif nx.is_empty(G):
        numero_cromatico_teorico = 1
    else:
        numero_cromatico_teorico = None
    
    return {
        'nombre': nombre,
        'nodos': n,
        'aristas': m,
        'densidad': densidad,
        'grado_promedio': grado_promedio,
        'grado_max': grado_max,
        'grado_min': grado_min,
        'componentes': num_componentes,
        'clustering': clustering_promedio,
        'cota_superior': cota_superior_cromatica,
        'chi_teorico': numero_cromatico_teorico
    }

## **7. FUNCIÓN PRINCIPAL DE CARGA**

In [8]:
def cargar_benchmark(fuente, **kwargs):
    """
    Función unificada para cargar benchmarks.
    
    Fuentes:
    - 'suite': Suite predefinida (nivel='simple'|'medio'|'complejo'|'completo')
    - 'dimacs': Archivo DIMACS (filepath='...')
    - 'snap': Archivo SNAP (filepath='...', directed=False)
    - 'simple': Grafo simple (tipo='...', n=..., ...)
    - 'complejo': Grafo complejo (tipo='...', n=..., ...)
    """
    if fuente == 'suite':
        nivel = kwargs.get('nivel', 'simple')
        return cargar_suite_benchmarks(nivel)
    
    elif fuente == 'dimacs':
        filepath = kwargs.get('filepath')
        G = cargar_dimacs(filepath)
        nombre = kwargs.get('nombre', filepath.split('/')[-1])
        return [(G, nombre)] if G else []
    
    elif fuente == 'snap':
        filepath = kwargs.get('filepath')
        directed = kwargs.get('directed', False)
        G = cargar_snap(filepath, directed)
        nombre = kwargs.get('nombre', filepath.split('/')[-1])
        return [(G, nombre)] if G else []
    
    elif fuente == 'simple':
        tipo = kwargs.get('tipo', 'random')
        n = kwargs.get('n', 50)
        return [generar_grafo_simple(tipo, n, **kwargs)]
    
    elif fuente == 'complejo':
        tipo = kwargs.get('tipo', 'scale_free')
        n = kwargs.get('n', 100)
        return [generar_benchmark_complejo(tipo, n, **kwargs)]
    
    else:
        raise ValueError(f"Fuente desconocida: {fuente}")

## **8. EJEMPLO DE USO**

In [9]:
print("="*80)
print("CARGANDO SUITE DE BENCHMARKS")
print("="*80)

benchmarks = cargar_benchmark('suite', nivel='simple')

print(f"\nTotal de benchmarks cargados: {len(benchmarks)}\n")

estadisticas = []
for G, nombre in benchmarks:
    stats = estadisticas_grafo(G, nombre)
    if stats:
        estadisticas.append(stats)

df_stats = pd.DataFrame(estadisticas)
print(df_stats.to_string(index=False))
print("\n" + "="*80)

CARGANDO SUITE DE BENCHMARKS


AttributeError: nx.random_tree was removed in version 3.4. Use `nx.random_labeled_tree` instead.
See: https://networkx.org/documentation/latest/release/release_3.4.html